### Подготовка датасета
+ Прочитаем датасет землетрясений Японии из csv
+ Уменьшим немного область, убрав землетрясения далекие от острова (следуя статье Подход к автоматическому прогнозу землетрясений В.Г. Гитис, А.Б. Дерендяев)
+ Перенсем координаты в (0;0)
+ Перейдем от абсолютных дат землетрясений в формте год-месяц-день время к вещественному числу дней после первого землетрясения в датасете 

In [1]:
import numpy as np
import pandas as pd
from functools import partial

In [2]:
def concatenate_time(a, b, c, sep):
    return(str(a) + sep + str(b) + sep + str(c))

In [3]:
make_date = partial(concatenate_time, sep='/')
make_time = partial(concatenate_time, sep=':')

earthquakers = pd.read_csv('data/90-16-jap.csv')
date = map(make_date, earthquakers['year'], earthquakers['month'], earthquakers['date'])
time = map(make_time, earthquakers['hour'], earthquakers['minute'], earthquakers['seconds'].astype(int))
earthquakers['date_time'] = pd.to_datetime(map(lambda date,time: date + ' ' + time, date, time))
earthquakers['diff_in_days_before_first_eqarthquake']\
= np.array([float(i) for i in earthquakers.date_time.values - earthquakers.date_time.values[0]]) / 1e+9/60./60./ 24.

data = earthquakers[['lat', 'long', 'depth', 'class', 'date_time', 'diff_in_days_before_first_eqarthquake']]
#  как в статье Подход к автоматическому прогнозу землетрясений В.Г. Гитис, А.Б. Дерендяев
#  [0]lat 30 : 50
#  [1]lon 110 : 150
lat_flag = (data['lat'].values <= 50.0) * (data['lat'].values >= 30.0)
lot_flag = (data['long'].values <= 150.0) * (data['long'].values >= 110.0)
data = data.ix[lat_flag * lot_flag,:]
l_from_magnitude = lambda M: np.exp(0.5*M-1.8)
l_from_magnitude = np.vectorize(l_from_magnitude)
data['l_r'] = l_from_magnitude(data['class'].values)




#  сдвинем координаты к 0
data['lat'] = data['lat'] - data['lat'].min()
data['long'] = data['long'] - data['long'].min()

pd.to_pickle(data, 'work_catalog_pd')

In [4]:
data.head(n=5)

,lat,long,depth,class,date_time,diff_in_days_before_first_eqarthquake,l_r
0,12.2734,20.5460,8.0,2.9,1990-01-01 02:19:05,0.000000,0.704688
1,9.9358,18.2461,30.6,3.0,1990-01-01 04:04:20,0.073090,0.740818
2,6.7053,16.9990,18.6,2.7,1990-01-01 07:14:11,0.204931,0.637628
3,6.4859,16.3964,67.2,4.3,1990-01-01 09:03:12,0.280637,1.419068
4,5.0121,13.8440,22.6,2.5,1990-01-01 12:31:25,0.425231,0.576950
